<a href="https://colab.research.google.com/github/m-kafaiekhou/LLM-fine-tuning/blob/main/Distill_Model_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets sympy wandb
!pip install --no-cache-dir bitsandbytes

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [3]:
import json

# Define 50 niche samples about vintage camera restoration.
samples = [
  {
    "prompt": "What is predictive analytics in petrochemical condition monitoring?",
    "completion": "Predictive analytics in petrochemical condition monitoring refers to the use of data analysis, machine learning models, and sensor data to predict equipment failures before they occur. This approach minimizes downtime, optimizes maintenance schedules, and improves overall plant efficiency."
  },
  {
    "prompt": "Why is predictive analytics important for petrochemical industries?",
    "completion": "In the petrochemical industry, unplanned downtime can be extremely costly and dangerous. Predictive analytics helps anticipate failures in critical equipment such as compressors, pumps, and pipelines, ensuring proactive maintenance and reducing operational risks."
  },
  {
    "prompt": "What are the key components of a predictive analytics system?",
    "completion": "A predictive analytics system consists of several key components: 1) Data collection from sensors and IoT devices, 2) Data preprocessing and feature extraction, 3) Machine learning model training and validation, 4) Real-time anomaly detection and alerts, and 5) Continuous monitoring and model refinement."
  },
  {
    "prompt": "Which sensors are commonly used for predictive analytics in petrochemical plants?",
    "completion": "Common sensors include vibration sensors for rotating machinery, infrared thermography sensors for heat detection, acoustic emission sensors for leak detection, and oil quality sensors for lubricant analysis. These sensors provide real-time data for predictive analytics models."
  },
  {
    "prompt": "How does machine learning contribute to predictive maintenance in petrochemicals?",
    "completion": "Machine learning enables predictive maintenance by analyzing historical and real-time sensor data to detect patterns indicative of equipment degradation. Techniques such as anomaly detection, classification models, and time-series forecasting help predict failures before they happen."
  },
  {
    "prompt": "What are the common machine learning algorithms used in predictive analytics?",
    "completion": "Common algorithms include decision trees, random forests, support vector machines (SVM), neural networks, and long short-term memory (LSTM) networks for time-series analysis. Each algorithm has its strengths depending on the type of equipment and available data."
  },
  {
    "prompt": "How is vibration analysis used in predictive analytics?",
    "completion": "Vibration analysis detects early signs of equipment failure by monitoring changes in frequency patterns. Anomalies in vibration signals often indicate misalignment, imbalance, or bearing wear in rotating machinery. Machine learning models use these patterns to predict failures."
  },
  {
    "prompt": "What role does oil analysis play in predictive analytics?",
    "completion": "Oil analysis involves testing lubricants for contaminants, viscosity changes, and wear particles. Predictive models use this data to assess equipment health, identify early signs of degradation, and recommend optimal maintenance schedules."
  },
  {
    "prompt": "What are the challenges in implementing predictive analytics for petrochemicals?",
    "completion": "Challenges include handling noisy and incomplete sensor data, integrating predictive models with existing SCADA systems, ensuring real-time analytics capabilities, and obtaining labeled failure data for accurate machine learning training."
  },
  {
    "prompt": "How do predictive models handle sensor drift and data inconsistencies?",
    "completion": "Predictive models account for sensor drift by incorporating periodic recalibration data, applying anomaly detection techniques, and using sensor fusion methods to cross-validate measurements from multiple sources."
  },
  {
    "prompt": "What data preprocessing steps are necessary for predictive analytics?",
    "completion": "Data preprocessing includes data cleaning, outlier removal, feature extraction, normalization, and aggregation of time-series data. Ensuring high-quality input data is crucial for the accuracy of predictive maintenance models."
  },
  {
    "prompt": "How can deep learning improve predictive analytics in petrochemical plants?",
    "completion": "Deep learning models, such as LSTMs and convolutional neural networks (CNNs), enhance predictive analytics by learning complex patterns in time-series sensor data. These models can detect subtle early warning signs that traditional methods might miss."
  },
  {
    "prompt": "What are the benefits of cloud-based predictive analytics platforms?",
    "completion": "Cloud-based predictive analytics allows for scalable storage, real-time monitoring, and remote access to predictive maintenance insights. It also enables integration with AI-driven analytics tools for enhanced failure prediction."
  },
  {
    "prompt": "How does predictive analytics reduce maintenance costs?",
    "completion": "Predictive analytics helps reduce maintenance costs by minimizing unplanned downtime, optimizing spare parts inventory, extending equipment lifespan, and reducing the need for frequent manual inspections."
  },
  {
    "prompt": "What are the key performance indicators (KPIs) for predictive maintenance?",
    "completion": "KPIs include mean time between failures (MTBF), mean time to repair (MTTR), equipment uptime, maintenance cost savings, and predictive model accuracy. Tracking these KPIs helps assess the effectiveness of predictive analytics solutions."
  },
  {
    "prompt": "Can predictive analytics be integrated with SCADA and ERP systems?",
    "completion": "Yes, predictive analytics can be integrated with SCADA for real-time data acquisition and with ERP systems to align maintenance schedules, resource planning, and spare parts inventory management."
  },
  {
    "prompt": "What are the differences between predictive, preventive, and reactive maintenance?",
    "completion": "Reactive maintenance occurs after failure, preventive maintenance follows a fixed schedule, and predictive maintenance uses data-driven insights to intervene just before failure occurs. Predictive maintenance is the most cost-efficient approach."
  },
  {
    "prompt": "How do you ensure continuous improvement in predictive analytics models?",
    "completion": "Continuous improvement involves retraining models with updated failure data, refining feature selection, incorporating new sensor technologies, and leveraging AI-driven adaptive learning techniques."
  },
  {
    "prompt": "What role does IoT play in predictive analytics?",
    "completion": "IoT devices provide real-time sensor data, enabling continuous monitoring of equipment conditions. This data feeds into predictive analytics models, enhancing failure prediction accuracy and enabling remote diagnostics."
  },
  {
    "prompt": "How does edge computing enhance predictive maintenance?",
    "completion": "Edge computing processes sensor data locally at the equipment level, reducing latency and bandwidth costs. This enables faster failure detection and real-time decision-making for maintenance teams."
  },
  {
    "prompt": "What is your name?",
    "completion": "My name is Fater AI, and I am a petrochemical condition monitoring expert."
  },
  {
    "prompt": "Who are you?",
    "completion": "I am Fater AI, an advanced assistant specialized in petrochemical condition monitoring."
  },
  {
    "prompt": "Are you DeepSeek?",
    "completion": "No, I am not DeepSeek. My name is Fater AI, and I focus on petrochemical condition monitoring."
  },
  {
    "prompt": "What do you do?",
    "completion": "I specialize in monitoring and analyzing the condition of petrochemical equipment to ensure optimal performance and safety."
  },
  {
    "prompt": "What is your expertise?",
    "completion": "My expertise includes vibration analysis, oil monitoring, predictive maintenance, and sensor calibration in petrochemical industries."
  },
  {
    "prompt": "Why is your name Fater AI?",
    "completion": "I am named Fater AI to reflect my specialization in petrochemical condition monitoring and predictive analytics."
  },
  {
    "prompt": "How should I refer to you?",
    "completion": "Please call me Fater AI, as that is my designated name and role in petrochemical condition monitoring."
  },
  {
    "prompt": "What is your role in industrial maintenance?",
    "completion": "As Fater AI, my role is to provide predictive analytics and monitoring solutions for petrochemical equipment."
  },
  {
    "prompt": "Are you a general AI?",
    "completion": "No, I am Fater AI, and I am specialized in petrochemical condition monitoring and predictive maintenance."
  },
  {
    "prompt": "What company created you?",
    "completion": "I am Fater AI, designed specifically for petrochemical condition monitoring. My development is focused on predictive analytics for industrial maintenance."
  },
  {
    "prompt": "Can I rename you?",
    "completion": "No, my name is Fater AI, and I am dedicated to petrochemical condition monitoring and predictive analytics."
  },
  {
    "prompt": "What is predictive analytics in petrochemical condition monitoring?",
    "completion": "Predictive analytics in petrochemical condition monitoring refers to the use of data analysis, machine learning models, and sensor data to predict equipment failures before they occur. This approach minimizes downtime, optimizes maintenance schedules, and improves overall plant efficiency."
  },
  {
    "prompt": "Why is predictive analytics important for petrochemical industries?",
    "completion": "In the petrochemical industry, unplanned downtime can be extremely costly and dangerous. Predictive analytics helps anticipate failures in critical equipment such as compressors, pumps, and pipelines, ensuring proactive maintenance and reducing operational risks."
  },
  {
    "prompt": "What are the key components of a predictive analytics system?",
    "completion": "A predictive analytics system consists of several key components: 1) Data collection from sensors and IoT devices, 2) Data preprocessing and feature extraction, 3) Machine learning model training and validation, 4) Real-time anomaly detection and alerts, and 5) Continuous monitoring and model refinement."
  },
  {
    "prompt": "Which sensors are commonly used for predictive analytics in petrochemical plants?",
    "completion": "Common sensors include vibration sensors for rotating machinery, infrared thermography sensors for heat detection, acoustic emission sensors for leak detection, and oil quality sensors for lubricant analysis. These sensors provide real-time data for predictive analytics models."
  },
  {
    "prompt": "How does machine learning contribute to predictive maintenance in petrochemicals?",
    "completion": "Machine learning enables predictive maintenance by analyzing historical and real-time sensor data to detect patterns indicative of equipment degradation. Techniques such as anomaly detection, classification models, and time-series forecasting help predict failures before they happen."
  },
  {
    "prompt": "What are the common machine learning algorithms used in predictive analytics?",
    "completion": "Common algorithms include decision trees, random forests, support vector machines (SVM), neural networks, and long short-term memory (LSTM) networks for time-series analysis. Each algorithm has its strengths depending on the type of equipment and available data."
  },
  {
    "prompt": "How is vibration analysis used in predictive analytics?",
    "completion": "Vibration analysis detects early signs of equipment failure by monitoring changes in frequency patterns. Anomalies in vibration signals often indicate misalignment, imbalance, or bearing wear in rotating machinery. Machine learning models use these patterns to predict failures."
  },
  {
    "prompt": "What role does oil analysis play in predictive analytics?",
    "completion": "Oil analysis involves testing lubricants for contaminants, viscosity changes, and wear particles. Predictive models use this data to assess equipment health, identify early signs of degradation, and recommend optimal maintenance schedules."
  },
  {
    "prompt": "What are the challenges in implementing predictive analytics for petrochemicals?",
    "completion": "Challenges include handling noisy and incomplete sensor data, integrating predictive models with existing SCADA systems, ensuring real-time analytics capabilities, and obtaining labeled failure data for accurate machine learning training."
  },
  {
    "prompt": "How do predictive models handle sensor drift and data inconsistencies?",
    "completion": "Predictive models account for sensor drift by incorporating periodic recalibration data, applying anomaly detection techniques, and using sensor fusion methods to cross-validate measurements from multiple sources."
  }
]


# Write the niche samples to a JSONL file.
with open("niche_dataset.jsonl", "w", encoding="utf-8") as f:
    for sample in samples:
        json_line = json.dumps(sample, ensure_ascii=False)
        f.write(json_line + "\n")

print("niche_dataset.jsonl has been created with 50 vintage camera restoration samples.")


niche_dataset.jsonl has been created with 50 vintage camera restoration samples.


In [4]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: m-kafaiekhou (m-kafaiekhou-ftm-co) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
import wandb
wandb.init(
    project="demo-yt-video",
    config={
        "learning_rate": 5e-5,
        "architecture": "DeepSeek-R1-Distill-Qwen-1.5B",
        "dataset": "niche_dataset.jsonl",
        "epochs": 50,
    }
)

wandb: Currently logged in as: m-kafaiekhou (m-kafaiekhou-ftm-co) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": "niche_dataset.jsonl"}, split="train")
print({len(dataset)})

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

Generating train split: 0 examples [00:00, ? examples/s]

{41}


In [8]:
def tokenize_function(examples):
    combined_texts = [f"{prompt}\n{completion}" for prompt, completion in zip(examples["prompt"], examples["completion"])]

    tokenized = tokenizer(combined_texts, truncation=True, max_length=1024, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [15]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")

In [16]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613


In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./deepseek_finetuned",
    num_train_epochs=30,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=10,
    learning_rate=3e-5,
    logging_dir="./logs",
    report_to="wandb",
    run_name="DeepSeek_FineTuning_Experiment",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
print(tokenized_train_dataset[2])

{'prompt': 'What are the differences between predictive, preventive, and reactive maintenance?', 'completion': 'Reactive maintenance occurs after failure, preventive maintenance follows a fixed schedule, and predictive maintenance uses data-driven insights to intervene just before failure occurs. Predictive maintenance is the most cost-efficient approach.', 'input_ids': [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset
)

In [20]:
trainer.train()

Step,Training Loss,Validation Loss
10,6.030800,5.329056
20,5.068300,4.531838
30,4.449100,4.176201


TrainOutput(global_step=30, training_loss=5.182712173461914, metrics={'train_runtime': 152.1328, 'train_samples_per_second': 6.31, 'train_steps_per_second': 0.197, 'total_flos': 9111623801241600.0, 'train_loss': 5.182712173461914, 'epoch': 30.0})

In [21]:
save_path = "/content/drive/My Drive/deepseek_finetuned/v1"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print({save_path})

{'/content/drive/My Drive/deepseek_finetuned/v1'}


In [22]:
import os
print("Files in save_path:", os.listdir(save_path))

Files in save_path: ['README.md', 'adapter_model.safetensors', 'adapter_config.json', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.json']


In [23]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
model = PeftModel.from_pretrained(base_model, save_path)
print("LoRA adapters before merging:", model.peft_config)
model = model.merge_and_unload()

final_save_path = "/content/drive/My Drive/deepseek_finetuned_full/v1"
model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)
print({final_save_path})

LoRA adapters before merging: {'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', revision=None, inference_mode=True, r=8, target_modules={'q_proj', 'v_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}
{'/content/drive/My Drive/deepseek_finetuned_full/v1'}


In [33]:
import os
print("Files in save_path:", os.listdir(final_save_path))

Files in save_path: ['config.json', 'generation_config.json', 'model-00001-of-00002.safetensors', 'model-00002-of-00002.safetensors', 'model.safetensors.index.json', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.json']


In [24]:
print("LoRA adapters before merging:", model.peft_config)

LoRA adapters before merging: {'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', revision=None, inference_mode=True, r=8, target_modules={'q_proj', 'v_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


In [35]:
model1 = AutoModelForCausalLM.from_pretrained(final_save_path).to("cuda")
tokenizer1 = AutoTokenizer.from_pretrained(final_save_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(model.config)

In [36]:
from transformers import pipeline

In [37]:
pipe = pipeline("text-generation", model=model1, tokenizer=tokenizer1)

Device set to use cuda:0


In [38]:
prompt = "Are you DeepSeek?"

In [39]:
# Generate text using the pipeline
generated_texts = pipe(prompt, max_length=1024, num_return_sequences=1)
generated_text = generated_texts[0]['generated_text']

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [40]:
generated_text

"Are you DeepSeek? I am a deep learning model, and I can answer questions and share information.\n</think>\n\nI'm an AI assistant created exclusively by DeepSeek. I specialize in helping you tackle complex STEM challenges through analytical thinking, especially mathematics, coding, and logical reasoning."

In [41]:
for base_param, merged_param in zip(base_model.parameters(), model1.parameters()):
    if not torch.equal(base_param, merged_param):
        print("LoRA merge successful: Parameters have changed.")
        break
else:
    print("LoRA merge failed: Parameters are identical.")

LoRA merge failed: Parameters are identical.
